In [1]:
# Load libraries
import pandas as pd
import numpy as np
from matplotlib.legend_handler import HandlerLine2D
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn import preprocessing
from IPython.display import Image
import pydotplus
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from multiprocessing import Process, Manager
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
path_to_dataset="creditcard.csv"
col_names=["Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"]
feature_cols = ["Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount"]
target=["Class"]
prob_data = pd.read_csv(path_to_dataset, header=None, names=col_names)
prob_data.head()
#split dataset in features and target variable
X = prob_data[feature_cols] # Features
y = prob_data[target] # Target variable


In [2]:
print('No Frauds', round(prob_data['Class'].value_counts()[0]/len(prob_data) * 100,2), '% of the dataset')
print('Frauds', round(prob_data['Class'].value_counts()[1]/len(prob_data) * 100,2), '% of the dataset')

No Frauds 99.83 % of the dataset
Frauds 0.17 % of the dataset


In [3]:
# Definition of personal functions
def report_metric(test, pred):
    print("Balanced accuracy of neural network is \n",
        metrics.balanced_accuracy_score(test, pred))
    print("Average_precision_score of neural network is \n",
        metrics.average_precision_score(test, pred))
    print('Confusion Matrix:\n', confusion_matrix(test, pred))
    print('Classification Report:\n', classification_report(test, pred))
        
def print_train_test_stats(y_train, y_pred_train, y_test, y_pred_test):
    '''Print Statistics of recent predictions on training & test data'''
    print('Training Error Statistics:')
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_train, y_pred_train))
    print('Mean Absolute Error:', np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)))
    print('Confusion Matrix\n', confusion_matrix(y_train, y_pred_train))
    print('Classification Report:\n', classification_report(y_train, y_pred_train))
    print('Accuracy Score:', accuracy_score(y_train, y_pred_train))

    print('\nTest Error Statistics:')
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_test))
    print('Mean Absolute Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_test)))
    print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred_test))
    print('Classification Report:\n', classification_report(y_test, y_pred_test))
    print('Accuracy Score:', accuracy_score(y_test, y_pred_test))


In [4]:
#feature scaling, Fit only to the feature X data
scaler = StandardScaler()

# Create Dataset Oversampling=False, normalization=False, standardization=True
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8,
                                                    random_state=7,shuffle=True,stratify=y) # 80% training and 20% test
# Scale input feature values
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
# Imbalanced Base Model with Standardized data with Standard Scaler
print('\nImbalanced dataset Base Model Results: \n')

# Create NN and fit and test it
mlp = MLPClassifier(hidden_layer_sizes=(15,), random_state=7, max_iter=1000,
                          tol=1e-6,learning_rate='adaptive')
clf = mlp.fit(X_train,y_train.values.ravel())
y_train_pred = clf.predict(X_train)
y_train_score = clf.predict_proba(X_train)
print("Training Data Performance Metrics:")
report_metric(test=y_train, pred=y_train_pred)

y_test_pred = clf.predict(X_test)
y_test_score = clf.predict_proba(X_test)
print("Test Data Performance Metrics:")
report_metric(test=y_test, pred=y_test_pred)


Imbalanced dataset Base Model Results: 

Training Data Performance Metrics:
Balanced accuracy of neural network is 
 0.9365108526638202
Average_precision_score of neural network is 
 0.8322005489963944
Confusion Matrix:
 [[227434     17]
 [    50    344]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    227451
           1       0.95      0.87      0.91       394

    accuracy                           1.00    227845
   macro avg       0.98      0.94      0.96    227845
weighted avg       1.00      1.00      1.00    227845

Test Data Performance Metrics:
Balanced accuracy of neural network is 
 0.8876759658562355
Average_precision_score of neural network is 
 0.6937835813374541
Confusion Matrix:
 [[56855     9]
 [   22    76]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.89      0.78      0.83        9

In [63]:
# GridSearch - Solver tuning
from sklearn.metrics import average_precision_score

mlp = MLPClassifier(max_iter=1000,random_state=7)
parameter_space = {
    'hidden_layer_sizes': [(15,)],
    'solver': ['sgd', 'adam','lbfgs'],
    'learning_rate': ['adaptive'],
    'tol':[1e-6],
    'max_iter':[1000]
}
clf = GridSearchCV(mlp, parameter_space, cv=5,scoring='average_precision').fit(X_train, y_train.values.ravel())
# Best paramete set
print('Best parameters found:\n', clf.best_params_)
pd.DataFrame( clf.cv_results_)
#Winner: sgd

C:\Users\Mahdi\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Best parameters found:
 {'hidden_layer_sizes': (15,), 'learning_rate': 'adaptive', 'max_iter': 1000, 'solver': 'sgd', 'tol': 1e-06}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,param_learning_rate,param_max_iter,param_solver,param_tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,429.829195,38.393291,0.038177,0.001599,"(15,)",adaptive,1000,sgd,1e-06,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.768687,0.855581,0.870961,0.822733,0.844334,0.832459,0.035551,1
1,120.911933,18.716969,0.042776,0.005073,"(15,)",adaptive,1000,adam,1e-06,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.730653,0.809608,0.845591,0.766417,0.820238,0.794502,0.040905,2
2,135.692247,29.844760,0.039377,0.003496,"(15,)",adaptive,1000,lbfgs,1e-06,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.630014,0.732558,0.708033,0.685999,0.781082,0.707537,0.050018,3


In [7]:

parameter_space = {
    'hidden_layer_sizes': [(15,)],
    'solver': ['sgd'],
    'learning_rate': ['adaptive'],
    'learning_rate_init':1e-9*np.logspace(0,9, base=10, num=10),
    'tol':[1e-6],
    'max_iter':[1000]
}
clf = GridSearchCV(mlp, parameter_space, cv=5,scoring='average_precision',n_jobs=-1).fit(X_train, y_train.values.ravel())
# Best paramete set
print('Best parameters found:\n', clf.best_params_)
pd.DataFrame( clf.cv_results_)
#winner learning_rate_init 0.001

Best parameters found:
 {'hidden_layer_sizes': (15,), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 1000, 'solver': 'sgd', 'tol': 1e-06}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,param_learning_rate,param_learning_rate_init,param_max_iter,param_solver,param_tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,898.110707,5.512033,0.065538,6.295009e-03,"(15,)",adaptive,1e-09,1000,sgd,1e-06,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.000936,0.000933,0.000916,0.000938,0.000914,0.000927,0.000010,8
1,891.100987,2.638802,0.065623,6.249166e-03,"(15,)",adaptive,1e-08,1000,sgd,1e-06,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.000936,0.000928,0.000916,0.000935,0.000913,0.000925,0.000010,10
2,896.706848,3.771246,0.054940,7.266734e-03,"(15,)",adaptive,1e-07,1000,sgd,1e-06,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.000961,0.000917,0.000915,0.000926,0.000909,0.000925,0.000019,9
3,892.716282,6.406984,0.051910,6.176784e-03,"(15,)",adaptive,1e-06,1000,sgd,1e-06,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.004356,0.004362,0.001493,0.001331,0.001546,0.002618,0.001424,7
4,882.914603,3.013624,0.054900,7.000545e-03,"(15,)",adaptive,1e-05,1000,sgd,1e-06,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.565702,0.741184,0.601180,0.665063,0.689642,0.652554,0.062561,6
5,593.179903,12.296019,0.062498,5.917394e-07,"(15,)",adaptive,0.0001,1000,sgd,1e-06,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.671513,0.798743,0.697694,0.746926,0.787637,0.740503,0.049492,4
6,657.751229,78.099763,0.061145,2.703810e-03,"(15,)",adaptive,0.001,1000,sgd,1e-06,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.768687,0.855581,0.870961,0.822733,0.844334,0.832459,0.035551,1
7,401.198481,22.487261,0.056248,7.653857e-03,"(15,)",adaptive,0.01,1000,sgd,1e-06,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.767047,0.842030,0.866518,0.808547,0.851865,0.827201,0.035610,2
8,299.971392,40.558070,0.062498,1.336846e-06,"(15,)",adaptive,0.1,1000,sgd,1e-06,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.743968,0.802549,0.832037,0.795890,0.856059,0.806101,0.037804,3
9,317.926548,62.831527,0.056249,1.593247e-02,"(15,)",adaptive,1,1000,sgd,1e-06,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.625627,0.710657,0.685928,0.728472,0.715672,0.693271,0.036532,5


In [8]:
parameter_space = {
    'hidden_layer_sizes': [(15,)],
    'solver': ['sgd'],
    'learning_rate': ['adaptive'],
    'learning_rate_init':[0.001],
    'tol':1e-9*np.logspace(0,9, base=10, num=10),
    'max_iter':[1000]
}
clf = GridSearchCV(mlp, parameter_space, cv=5,scoring='average_precision',n_jobs=-1).fit(X_train, y_train.values.ravel())
# Best paramete set
print('Best parameters found:\n', clf.best_params_)
pd.DataFrame( clf.cv_results_)
#Winner tol = 1e-09

Best parameters found:
 {'hidden_layer_sizes': (15,), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 1000, 'solver': 'sgd', 'tol': 1e-09}


C:\Users\Mahdi\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,param_learning_rate,param_learning_rate_init,param_max_iter,param_solver,param_tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,903.828072,5.208235,0.059374,0.006250,"(15,)",adaptive,0.001,1000,sgd,1e-09,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.782269,0.859837,0.873309,0.821744,0.848158,0.837063,0.032226,1
1,896.893599,1.805464,0.065623,0.006250,"(15,)",adaptive,0.001,1000,sgd,1e-08,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.782269,0.859837,0.873309,0.821744,0.848158,0.837063,0.032226,1
2,899.484856,2.646619,0.065623,0.006249,"(15,)",adaptive,0.001,1000,sgd,1e-07,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.782269,0.859837,0.873309,0.821744,0.848158,0.837063,0.032226,1
3,677.522236,80.226408,0.065582,0.006271,"(15,)",adaptive,0.001,1000,sgd,1e-06,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.768687,0.855581,0.870961,0.822733,0.844334,0.832459,0.035551,4
4,116.315645,0.912347,0.062499,0.009882,"(15,)",adaptive,0.001,1000,sgd,1e-05,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.659706,0.802280,0.707264,0.750535,0.776087,0.739174,0.050611,5
5,76.143769,0.854539,0.065623,0.006250,"(15,)",adaptive,0.001,1000,sgd,0.0001,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.627111,0.772467,0.649299,0.703828,0.739532,0.698447,0.054222,6
6,66.000006,0.236977,0.071833,0.007621,"(15,)",adaptive,0.001,1000,sgd,0.001,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.587721,0.762451,0.635892,0.690882,0.717509,0.678891,0.061283,7
7,63.040655,0.148495,0.068727,0.007671,"(15,)",adaptive,0.001,1000,sgd,0.01,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.582285,0.753098,0.624604,0.679937,0.708926,0.669770,0.060455,8
8,61.596953,0.208560,0.071833,0.007623,"(15,)",adaptive,0.001,1000,sgd,0.1,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.578214,0.747664,0.617841,0.677416,0.703163,0.664860,0.060332,9
9,57.125926,5.841209,0.045695,0.010159,"(15,)",adaptive,0.001,1000,sgd,1,"{'hidden_layer_sizes': (15,), 'learning_rate':...",0.578214,0.747664,0.617841,0.677416,0.703163,0.664860,0.060332,9


In [38]:
parameter_space = {
    'hidden_layer_sizes': [(15,)],
    'solver': ['sgd'],
    'learning_rate': ['adaptive'],
    'learning_rate_init':[0.001],
    'tol':[1e-09],
    'max_iter':[1000],
    'batch_size':np.geomspace(10, 300, num=10,dtype =int)
}
clf = GridSearchCV(mlp, parameter_space, cv=5,scoring='average_precision',n_jobs=-1).fit(X_train, y_train.values.ravel())
# Best paramete set
print('Best parameters found:\n', clf.best_params_)
pd.DataFrame( clf.cv_results_)
#Winner: batch_Size: 140

Best parameters found:
 {'batch_size': 140, 'hidden_layer_sizes': (15,), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 1000, 'solver': 'sgd', 'tol': 1e-09}


C:\Users\Mahdi\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_hidden_layer_sizes,param_learning_rate,param_learning_rate_init,param_max_iter,param_solver,param_tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,10936.965322,101.263503,0.059365,0.000489,10,"(15,)",adaptive,0.001,1000,sgd,1e-09,"{'batch_size': 10, 'hidden_layer_sizes': (15,)...",0.769597,0.846357,0.875124,0.818344,0.858719,0.833628,0.037006,5
1,8121.440026,125.287576,0.059747,0.007493,14,"(15,)",adaptive,0.001,1000,sgd,1e-09,"{'batch_size': 14, 'hidden_layer_sizes': (15,)...",0.771327,0.842162,0.871522,0.817437,0.854886,0.831467,0.034874,7
2,5649.339552,38.196997,0.061764,0.002636,21,"(15,)",adaptive,0.001,1000,sgd,1e-09,"{'batch_size': 21, 'hidden_layer_sizes': (15,)...",0.761137,0.850252,0.865844,0.810591,0.853070,0.828179,0.038287,10
3,3962.980998,15.640444,0.059964,0.001672,31,"(15,)",adaptive,0.001,1000,sgd,1e-09,"{'batch_size': 31, 'hidden_layer_sizes': (15,)...",0.763424,0.852158,0.863542,0.813184,0.852039,0.828869,0.036906,9
4,2864.062196,15.303895,0.059166,0.001599,45,"(15,)",adaptive,0.001,1000,sgd,1e-09,"{'batch_size': 45, 'hidden_layer_sizes': (15,)...",0.767479,0.850054,0.868639,0.813954,0.850806,0.830187,0.036039,8
5,2070.305042,12.750780,0.062364,0.004583,66,"(15,)",adaptive,0.001,1000,sgd,1e-09,"{'batch_size': 66, 'hidden_layer_sizes': (15,)...",0.777179,0.857830,0.870745,0.814008,0.850846,0.834122,0.034147,4
6,1525.894280,2.943142,0.068359,0.010885,96,"(15,)",adaptive,0.001,1000,sgd,1e-09,"{'batch_size': 96, 'hidden_layer_sizes': (15,)...",0.786011,0.861167,0.872874,0.816537,0.851892,0.837696,0.031966,2
7,1175.674054,5.238803,0.060764,0.001165,140,"(15,)",adaptive,0.001,1000,sgd,1e-09,"{'batch_size': 140, 'hidden_layer_sizes': (15,...",0.785062,0.861017,0.871522,0.820544,0.855979,0.838825,0.031876,1
8,913.332935,3.556052,0.062762,0.001937,205,"(15,)",adaptive,0.001,1000,sgd,1e-09,"{'batch_size': 205, 'hidden_layer_sizes': (15,...",0.782195,0.859409,0.873001,0.821818,0.848874,0.837060,0.032166,3
9,708.385789,32.369040,0.054968,0.012106,300,"(15,)",adaptive,0.001,1000,sgd,1e-09,"{'batch_size': 300, 'hidden_layer_sizes': (15,...",0.774289,0.854808,0.866462,0.822477,0.842270,0.832061,0.032356,6


In [46]:
parameter_space = {
    'hidden_layer_sizes': [(15,)],
    'solver': ['sgd'],
    'learning_rate': ['adaptive'],
    'learning_rate_init':[0.001],
    'tol':[1e-09],
    'max_iter':[1000],
    'batch_size':[140],
    'momentum':np.geomspace(0.1, .9, num=10,dtype =float)
}
clf = GridSearchCV(mlp, parameter_space, cv=5,scoring='average_precision',n_jobs=-1).fit(X_train, y_train.values.ravel())
# Best paramete set
print('Best parameters found:\n', clf.best_params_)
pd.DataFrame( clf.cv_results_)
#Winner Momentum=0.9

Best parameters found:
 {'batch_size': 140, 'hidden_layer_sizes': (15,), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 1000, 'momentum': 0.9, 'solver': 'sgd', 'tol': 1e-09}


C:\Users\Mahdi\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_hidden_layer_sizes,param_learning_rate,param_learning_rate_init,param_max_iter,param_momentum,...,param_tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1191.142095,9.587829,0.057746,5.739536e-03,140,"(15,)",adaptive,0.001,1000,0.1,...,1e-09,"{'batch_size': 140, 'hidden_layer_sizes': (15,...",0.707610,0.820039,0.745317,0.786957,0.810470,0.774079,0.042066,10
1,1176.215860,6.715508,0.060157,7.859087e-03,140,"(15,)",adaptive,0.001,1000,0.127652,...,1e-09,"{'batch_size': 140, 'hidden_layer_sizes': (15,...",0.709829,0.821766,0.749274,0.788527,0.811511,0.776181,0.041485,9
2,1172.591390,12.822918,0.061763,6.108700e-03,140,"(15,)",adaptive,0.001,1000,0.16295,...,1e-09,"{'batch_size': 140, 'hidden_layer_sizes': (15,...",0.712446,0.822649,0.752562,0.791027,0.813748,0.778487,0.040950,8
3,1155.952306,2.916872,0.058564,1.020030e-03,140,"(15,)",adaptive,0.001,1000,0.208008,...,1e-09,"{'batch_size': 140, 'hidden_layer_sizes': (15,...",0.718530,0.824487,0.756107,0.793129,0.815370,0.781525,0.039345,7
4,1150.888376,1.036745,0.058965,8.313940e-07,140,"(15,)",adaptive,0.001,1000,0.265526,...,1e-09,"{'batch_size': 140, 'hidden_layer_sizes': (15,...",0.721421,0.827629,0.770070,0.794745,0.817765,0.786326,0.038058,6
5,1150.917756,1.311082,0.057254,5.366897e-03,140,"(15,)",adaptive,0.001,1000,0.338949,...,1e-09,"{'batch_size': 140, 'hidden_layer_sizes': (15,...",0.727837,0.829078,0.779488,0.799382,0.819178,0.790992,0.035878,5
6,1152.311192,0.845783,0.060577,1.609578e-03,140,"(15,)",adaptive,0.001,1000,0.432675,...,1e-09,"{'batch_size': 140, 'hidden_layer_sizes': (15,...",0.733995,0.831977,0.792328,0.803579,0.821800,0.796736,0.034272,4
7,1152.317571,0.856938,0.058666,6.052975e-03,140,"(15,)",adaptive,0.001,1000,0.552317,...,1e-09,"{'batch_size': 140, 'hidden_layer_sizes': (15,...",0.748770,0.837787,0.810566,0.811319,0.827587,0.807206,0.030963,3
8,1159.184871,3.397367,0.069158,1.370871e-02,140,"(15,)",adaptive,0.001,1000,0.705043,...,1e-09,"{'batch_size': 140, 'hidden_layer_sizes': (15,...",0.764818,0.848490,0.850786,0.821268,0.837761,0.824625,0.031669,2
9,983.279221,211.175497,0.051568,9.085146e-03,140,"(15,)",adaptive,0.001,1000,0.9,...,1e-09,"{'batch_size': 140, 'hidden_layer_sizes': (15,...",0.785062,0.861017,0.871522,0.820544,0.855979,0.838825,0.031876,1


In [54]:
parameter_space = {
    'hidden_layer_sizes': [(15,)],
    'solver': ['sgd'],
    'learning_rate': ['adaptive'],
    'learning_rate_init':[0.001],
    'tol':[1e-09],
    'max_iter':[1000],
    'batch_size':[140],
    'momentum':[0.9],
    'alpha':1e-9*np.logspace(0,9, base=10, num=10)
}
clf = GridSearchCV(mlp, parameter_space, cv=5,scoring='average_precision',n_jobs=-1).fit(X_train, y_train.values.ravel())
# Best paramete set
print('Best parameters found:\n', clf.best_params_)
pd.DataFrame( clf.cv_results_)
#Winner: alpha=0.01

Best parameters found:
 {'alpha': 0.01, 'batch_size': 140, 'hidden_layer_sizes': (15,), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 1000, 'momentum': 0.9, 'solver': 'sgd', 'tol': 1e-09}


C:\Users\Mahdi\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_batch_size,param_hidden_layer_sizes,param_learning_rate,param_learning_rate_init,param_max_iter,...,param_tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1147.479478,4.683272,0.058667,0.006054,1e-09,140,"(15,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 1e-09, 'batch_size': 140, 'hidden_la...",0.785209,0.860925,0.871506,0.820362,0.855957,0.838792,0.031829,7
1,1135.759051,5.778276,0.059965,0.000632,1e-08,140,"(15,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 1e-08, 'batch_size': 140, 'hidden_la...",0.785173,0.860924,0.871494,0.820359,0.855956,0.838781,0.031839,8
2,1126.402773,2.341898,0.058964,0.000632,1e-07,140,"(15,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 1.0000000000000001e-07, 'batch_size'...",0.785227,0.860975,0.871506,0.820375,0.855960,0.838809,0.031829,6
3,1127.456614,2.098302,0.058965,0.000632,1e-06,140,"(15,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 1.0000000000000002e-06, 'batch_size'...",0.785358,0.860975,0.871514,0.820360,0.855957,0.838833,0.031788,4
4,1126.921520,1.526232,0.059978,0.002089,1e-05,140,"(15,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 1e-05, 'batch_size': 140, 'hidden_la...",0.785474,0.860995,0.871519,0.820359,0.855958,0.838861,0.031753,3
5,1131.243955,2.661851,0.059365,0.000800,0.0001,140,"(15,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 0.0001, 'batch_size': 140, 'hidden_l...",0.785062,0.861017,0.871522,0.820544,0.855979,0.838825,0.031876,5
6,1130.115312,1.760106,0.059164,0.000747,0.001,140,"(15,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 0.001, 'batch_size': 140, 'hidden_la...",0.787867,0.862420,0.873356,0.821372,0.857006,0.840404,0.031523,2
7,1131.434177,2.187625,0.058166,0.000748,0.01,140,"(15,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 0.01, 'batch_size': 140, 'hidden_lay...",0.781963,0.870855,0.881658,0.833433,0.857525,0.845087,0.035430,1
8,1104.457294,3.246245,0.057472,0.002621,0.1,140,"(15,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 0.1, 'batch_size': 140, 'hidden_laye...",0.702916,0.823934,0.753503,0.785138,0.796813,0.772461,0.041471,9
9,915.168859,228.167420,0.048571,0.009129,1,140,"(15,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 1.0, 'batch_size': 140, 'hidden_laye...",0.675897,0.803932,0.727146,0.766574,0.749001,0.744510,0.042538,10


In [61]:
parameter_space = {
    'hidden_layer_sizes': [(5,),(10,),(15,),(20,),(25,),(30,),(35,),(40,),(45,),(50,)],
    'solver': ['sgd'],
    'learning_rate': ['adaptive'],
    'learning_rate_init':[0.001],
    'tol':[1e-09],
    'max_iter':[1000],
    'batch_size':[140],
    'momentum':[0.9],
    'alpha':[0.01]
}
clf = GridSearchCV(mlp, parameter_space, cv=5,scoring='average_precision',n_jobs=-1).fit(X_train, y_train.values.ravel())
# Best paramete set
print('Best parameters found:\n', clf.best_params_)
pd.DataFrame( clf.cv_results_)
#Winner hiden_layer_size:(50,)

Best parameters found:
 {'alpha': 0.01, 'batch_size': 140, 'hidden_layer_sizes': (30,), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 1000, 'momentum': 0.9, 'solver': 'sgd', 'tol': 1e-09}


C:\Users\Mahdi\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_batch_size,param_hidden_layer_sizes,param_learning_rate,param_learning_rate_init,param_max_iter,...,param_tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,961.306154,8.027383,0.038031,0.003640,0.01,140,"(5,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 0.01, 'batch_size': 140, 'hidden_lay...",0.737069,0.859752,0.863441,0.826156,0.826009,0.822485,0.045581,10
1,1021.691447,4.252178,0.050370,0.005348,0.01,140,"(10,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 0.01, 'batch_size': 140, 'hidden_lay...",0.770283,0.863186,0.878780,0.842914,0.842624,0.839557,0.037192,9
2,1119.677623,7.177081,0.065960,0.005895,0.01,140,"(15,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 0.01, 'batch_size': 140, 'hidden_lay...",0.781963,0.870855,0.881658,0.833433,0.857525,0.845087,0.035430,5
3,1218.624177,8.500989,0.072556,0.003609,0.01,140,"(20,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 0.01, 'batch_size': 140, 'hidden_lay...",0.777564,0.871154,0.875988,0.838754,0.854379,0.843568,0.035521,8
4,1308.970779,11.709915,0.087142,0.006145,0.01,140,"(25,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 0.01, 'batch_size': 140, 'hidden_lay...",0.772809,0.874641,0.885771,0.840120,0.856263,0.845921,0.039746,4
5,1404.295972,18.689429,0.097540,0.003927,0.01,140,"(30,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 0.01, 'batch_size': 140, 'hidden_lay...",0.781972,0.882309,0.890369,0.843466,0.866215,0.852866,0.038899,1
6,1523.858390,13.977304,0.112131,0.002134,0.01,140,"(35,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 0.01, 'batch_size': 140, 'hidden_lay...",0.778107,0.874894,0.881444,0.837395,0.852786,0.844925,0.036918,6
7,1609.621772,12.391986,0.121125,0.003368,0.01,140,"(40,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 0.01, 'batch_size': 140, 'hidden_lay...",0.777858,0.877245,0.887473,0.835636,0.860434,0.847729,0.039091,2
8,1689.407185,11.131309,0.132934,0.004831,0.01,140,"(45,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 0.01, 'batch_size': 140, 'hidden_lay...",0.777203,0.875329,0.882700,0.834006,0.849238,0.843695,0.037598,7
9,1498.187681,262.635596,0.118729,0.020628,0.01,140,"(50,)",adaptive,0.001,1000,...,1e-09,"{'alpha': 0.01, 'batch_size': 140, 'hidden_lay...",0.777300,0.872866,0.886262,0.841217,0.860190,0.847567,0.038143,3


In [62]:
parameter_space = {
    'hidden_layer_sizes': (50,),
    'random_state':7,
    'solver': 'sgd',
    'learning_rate': 'adaptive',
    'learning_rate_init':0.001,
    'tol':1e-09,
    'max_iter':1000,
    'batch_size':140,
    'momentum':0.9,
    'alpha':0.01
}
mlp.set_params(**parameter_space)
clf = mlp.fit(X_train,y_train.values.ravel())
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)
print("Training Data Performance Metrics:")
report_metric(test=y_train, pred=y_train_pred)
print("Test Data Performance Metrics:")
report_metric(test=y_test, pred=y_test_pred)

C:\Users\Mahdi\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Training Data Performance Metrics:
Balanced accuracy of neural network is 
 0.8984397866739725
Average_precision_score of neural network is 
 0.7563742422416538
Confusion Matrix:
 [[227434     17]
 [    80    314]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    227451
           1       0.95      0.80      0.87       394

    accuracy                           1.00    227845
   macro avg       0.97      0.90      0.93    227845
weighted avg       1.00      1.00      1.00    227845

Test Data Performance Metrics:
Balanced accuracy of neural network is 
 0.872431393773041
Average_precision_score of neural network is 
 0.725472902689393
Confusion Matrix:
 [[56862     2]
 [   25    73]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.97      0.74      0.84        98

    accuracy                           1.